# Settings

In [1]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import tensorflow as tf
import numpy.random as rnd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
%matplotlib inline
#from time import time
#from keras.callbacks import TensorBoard

/Users/mdeblaauw/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load data

In [2]:
import pickle
import boto3
from io import BytesIO

ModuleNotFoundError: No module named 'boto3'

In [ ]:
s3 = boto3.resource('s3')
data_subsets = ['train', 'val', 'test']
data = {}

for name in data_subsets:
    with BytesIO() as files:
        path = "omniglot_images/" +name+ ".pickle"
        s3.Bucket("research-paper-omniglot-data").download_fileobj(path, files)
        files.seek(0)    # move back to the beginning after writing
        (X,c) = pickle.load(files)
        data[name] = X

path = '../../omniglot_images/'
data_subsets = ["train", "val", "test"]

data = {}
categories = {}
info = {}
        
for name in data_subsets:
    file_path = os.path.join(path, name + ".pickle")
    print("loading data from {}".format(file_path))
    with open(file_path,"rb") as f:
        (X,c) = pickle.load(f)
        data[name] = X
        categories[name] = c

In [5]:
def create_train_data(size, s='train'):
    #get train data and shape
    X=data[s]
    n_classes, n_examples, w, h = X.shape
    
    #initialize 2 empty arrays for the input size in a list
    pairs=[np.zeros((size, h, w,1)) for i in range(2)]
    
    #initialize vector for the targets
    targets=np.zeros((size,1))
    
    for x in range(size):
        #randomly sample one class (character)
        category = rnd.choice(n_classes,1,replace=False)
        #randomly sample one example from class (1-20 characters)
        idx_1 = rnd.randint(0, n_examples)
        pairs[0][x,:,:,:] = X[category, idx_1].reshape(w, h, 1)
        #randomly sample again one example from class and add last class with modulo
        # ..to ensure not same class pairs are created
        idx_2 = (idx_1 + rnd.randint(0, n_examples)) % n_examples
        #pick images of different class for 1st half and same class for 2nd half
        if x >= size // 2:
            category_2 = category
            targets[x] = 1
        else: 
        #add a random number to the category modulo n classes to ensure 2nd image has
        # ..different category
            idx_2 = rnd.randint(0, n_examples) 
            category_2 = (category + rnd.randint(1,n_classes)) % n_classes
            targets[x] = 0
        pairs[1][x,:,:,:] = X[category_2,idx_2].reshape(w, h,1)
        
    return pairs, targets

In [6]:
train_set, train_labels = create_train_data(10000)
val_set, val_labels = create_train_data(3000, 'val')
test_set, test_labels = create_train_data(3000, 'test')

# Hyperparameter setting

In [7]:
learning_rate = 5.491e-05
reg_layer1 = 1.271e-06
reg_layer2 = 1e-08
reg_layer3 = 2.734e-05
reg_layer4 = 0.005639
reg_layer5 = 0.0004877
filt_layer1 = 5
filt_layer2 = 12
filt_layer3 = 7
filt_layer4 = 2
chan_layer1 = 64
chan_layer2 = 112
chan_layer3 = 96
chan_layer4 = 144
fc_layer5 = 1536
beta1 = 0.5745
beta2 = 0.9999
batch = 48

In [8]:
params = {}
params['learning_rate'] = learning_rate
params['reg_layer1'] = reg_layer1
params['reg_layer2'] = reg_layer2
params['reg_layer2'] = reg_layer2
params['reg_layer3'] = reg_layer3
params['reg_layer4'] = reg_layer4
params['reg_layer5'] = reg_layer5
params['filter_layer1'] = filt_layer1
params['filter_layer2'] = filt_layer2
params['filter_layer3'] = filt_layer3
params['filter_layer4'] = filt_layer4
params['channel_layer1'] = chan_layer1
params['channel_layer2'] = chan_layer2
params['channel_layer3'] = chan_layer3
params['channel_layer4'] = chan_layer4
params['channel_layer5'] = fc_layer5
params['beta1'] = beta1
params['beta2'] = beta2
params['batch'] = batch

# Create graph

In [9]:
def W_init(shape,name=None):
    """Initialize weights as in paper"""
    values = rnd.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values,name=name)

In [10]:
def b_init(shape,name=None):
    """Initialize bias as in paper"""
    values=rnd.normal(loc=0.5,scale=1e-2,size=shape)
    return K.variable(values,name=name)

In [11]:
def create_network(**params):
    input_shape = (105, 105, 1)
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    #build convnet to use in each siamese 'leg'
    convnet = Sequential()
    convnet.add(Conv2D(params['channel_layer1'],(params['filter_layer1'],params['filter_layer1']),activation='relu',input_shape=input_shape,kernel_initializer=W_init,kernel_regularizer=l2(params['reg_layer1']),bias_initializer=b_init))
    convnet.add(MaxPooling2D())
    convnet.add(Conv2D(params['channel_layer2'],(params['filter_layer2'],params['filter_layer2']),activation='relu',kernel_regularizer=l2(params['reg_layer2']),kernel_initializer=W_init,bias_initializer=b_init))
    convnet.add(MaxPooling2D())
    convnet.add(Conv2D(params['channel_layer3'],(params['filter_layer3'],params['filter_layer3']),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(params['reg_layer3']),bias_initializer=b_init))
    convnet.add(MaxPooling2D())
    convnet.add(Conv2D(params['channel_layer4'],(params['filter_layer4'],params['filter_layer4']),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(params['reg_layer4']),bias_initializer=b_init))
    convnet.add(Flatten())
    convnet.add(Dense(params['channel_layer5'],activation="sigmoid",kernel_regularizer=l2(params['reg_layer5']),kernel_initializer=W_init,bias_initializer=b_init))
    #call the convnet Sequential model on each of the input tensors so params will be shared
    encoded_l = convnet(left_input)
    encoded_r = convnet(right_input)
    #layer to merge two encoded inputs with the l1 distance between them
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    #call this layer on list of two input tensors.
    L1_distance = L1_layer([encoded_l, encoded_r])
    prediction = Dense(1,activation='sigmoid',bias_initializer=b_init)(L1_distance)
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    optimizer = Adam(lr=params['learning_rate'], beta_1=params['beta1'], beta_2=params['beta2'])
    siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)
    #siamese_net.count_params()
    return(siamese_net)

In [12]:
def accuracy(pred, true_val):
    acc_bool = np.equal(np.round_(pred), true_val)
    acc = np.mean(acc_bool.astype(int))
    
    return(acc)

# Training

This is just as fast as the Tensorflow implementation

In [21]:
###################################
# TensorFlow wizardry
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Create a session with the above options specified.
K.tensorflow_backend.set_session(tf.Session(config=config))
###################################


siamese_net = create_network(**params)
        
print("!")
batch_size = params['batch']
total_batch = int(10000/batch_size)
total_batch_val = int(3000/batch_size)
epoch = 10
summaries_dir = '../../logs-tensorboard/SCNN/'
train_writer = tf.summary.FileWriter(summaries_dir + '/train')
validation_writer = tf.summary.FileWriter(summaries_dir + '/val')
summary_op = tf.summary.merge_all()

print("training")
for i in range(epoch):
    train_batch_acc = 0
    val_batch_acc = 0
    for j in range(1+(total_batch*i), total_batch+(total_batch*i)):
        print(j)
        batch_x1, batch_x2, batch_y = shuffle(train_set[0],train_set[1], train_labels, n_samples = batch_size)
        loss=siamese_net.train_on_batch([batch_x1, batch_x2],batch_y)
        probs = siamese_net.predict([batch_x1, batch_x2])
        train_batch_acc1 = accuracy(probs, batch_y)
        train_batch_acc += train_batch_acc1
        
        summary_train = tf.Summary(value=[tf.Summary.Value(tag="summary", simple_value=train_batch_acc1),])
        
        train_writer.add_summary(summary_train, j)
        
        batch_x1, batch_x2, batch_y = shuffle(val_set[0],val_set[1], val_labels, n_samples = batch_size)
        probs = siamese_net.predict([batch_x1, batch_x2])
        val_batch_acc1 = accuracy(probs, batch_y)
        val_batch_acc += val_batch_acc1
        
        summary_val = tf.Summary(value=[tf.Summary.Value(tag="summary", simple_value= val_batch_acc1),])
        
        validation_writer.add_summary(summary_val, j)
        #print('Loss:', loss)
        #print('Batch:', j)
    train_acc = train_batch_acc/total_batch
    val_acc = val_batch_acc/total_batch      
    
    print('Train accuracy:', train_acc)
    print('Validation accuracy:', val_acc)
    summary_op = tf.summary.merge_all()
    
    if i == epoch-1:
        test_batch_acc = 0
        print('test start')
        for k in range(total_batch_val):
            batch_x1, batch_x2, batch_y = shuffle(test_set[0],test_set[1], test_labels, n_samples = batch_size)
            probs = siamese_net.predict([batch_x1, batch_x2])
            test_batch_acc += accuracy(probs, batch_y)
        test_acc = test_batch_acc/total_batch_val
        print('test:', test_acc)
    print('Epoch:', i)
    

!
training
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


KeyboardInterrupt: 

In [17]:
sorted(zip(search_result.func_vals, search_result.x_iters))

[(-0.7399193548387095,
  [3.2765338536539206e-07,
   4.8134311006773297e-05,
   0.0006242765412407358,
   1.037203178084381e-07,
   1e-08,
   3.9432125323605935e-08,
   11,
   4,
   5,
   3,
   192,
   48,
   112,
   64,
   3584,
   1e-05,
   0.5224310846332395,
   48]),
 (-0.719758064516129,
  [4.825880554313875e-07,
   2.9452322822804706e-06,
   0.0004939851502951343,
   2.2362840911377307e-07,
   1e-08,
   0.004050462111222351,
   11,
   3,
   4,
   4,
   160,
   256,
   176,
   112,
   1280,
   0.2507180516243929,
   0.8752709591633175,
   48]),
 (-0.7157258064516129,
  [1e-07,
   1.6063123836119204e-06,
   0.00041913458591738117,
   1.087258439661984e-05,
   1.4258996203117195e-08,
   0.00045690114319893937,
   12,
   7,
   8,
   5,
   240,
   256,
   144,
   160,
   1280,
   0.9999,
   0.07166075362238511,
   48]),
 (-0.7133736559139785,
  [3.633914639316309e-07,
   4.161396168274266e-05,
   0.0011259797494248196,
   2.2563351741885868e-07,
   1.4871600468513406e-08,
   0.0001036

In [18]:
search_result.x

[3.2765338536539206e-07,
 4.8134311006773297e-05,
 0.0006242765412407358,
 1.037203178084381e-07,
 1e-08,
 3.9432125323605935e-08,
 11,
 4,
 5,
 3,
 192,
 48,
 112,
 64,
 3584,
 1e-05,
 0.5224310846332395,
 48]